In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
from itertools import permutations
from collections import defaultdict

In [3]:
import sys
sys.path.append("../..")

In [4]:
from bdds.bdd_core import *
from bdds.node_bdds import *
from bdds.design_conditioned import *

In [5]:
from data_types import item_set

In [6]:
#helper = DesignInfo("../../encoding/dsl/rooms_for_alloy.txt", "../../encoding/dsl/exits_for_alloy.txt", "../../output/important_closure.dddmp")

In [7]:
import time

In [8]:
timea = time.time()

In [9]:
helper = NodeEnv("../../encoding/dsl/rooms_for_alloy.txt", "../../encoding/dsl/exits_for_alloy.txt")

In [10]:
start_bdd = helper.context.add_expr("node_id_prev = 0") & helper.mk_itemset_expr(item_set.ItemSet())

In [11]:
trans = helper.mk_trans()

In [12]:
trans.count()

158241627570176.0

In [13]:
%%capture
all_reachable_p = get_reachable_id(trans, helper.context, start_bdd)

In [14]:
# Feasible transitions only go between pairs of reachable nodes
trans_feasible = trans & all_reachable_p & (helper.context.let(prev_to_next, all_reachable_p))

In [15]:
trans_feasible.count()

7305430.0

In [16]:
# I don't think we need this...
#all_pairs_reachable = helper.mk_closure(trans_feasible)

In [17]:
get_E = helper.context.add_expr(f"E_prev = 0 & E_next = 1")
get_without_PB = helper.context.add_expr(f"PB_prev = 0") & get_E

In [18]:
itemset_lattice = helper.context.exist(pos_vars, trans_feasible)

In [19]:
trans_feasible

Function (DdNode) with var index: 19, ref count: 1, int repr: 96245159485379

In [20]:
#test = trans_feasible & helper.context.add_expr("Crocomire_prev = 0 & PB_prev = 0 & PB_next = 1")

In [21]:
#helper.context.pick(test)

In [22]:
(trans_feasible & helper.context.add_expr("Crocomire_prev = 0 & IB_prev = 0 & PB_next = 1")).count()

0.0

In [23]:
(trans_feasible & helper.context.add_expr("IB_prev = 0 & IB_next = 1")).count()

396.0

In [24]:
context = helper.context

In [25]:
get_pb = context.add_expr(f"PB_prev = 0 & PB_next = 1") & itemset_lattice

In [26]:
get_pb2 = context.let(prev_to_temp, get_pb)

In [27]:
# Compute the GTE relation on the sets
set_gte = context.true
for i in item_mapping:
    set_gte &= context.add_expr(f"{i}_temp >= {i}_prev")

In [28]:
set_eq = context.true
for i in item_mapping:
    set_eq &= context.add_expr(f"{i}_temp = {i}_prev")
set_ne = ~set_eq

In [29]:
set_lte = context.true
for i in item_mapping:
    set_lte &= context.add_expr(f"{i}_temp <= {i}_prev")
set_lt = set_lte & set_ne

In [30]:
def to_var(i, when="prev"):
    return context.add_expr(f"{i}_{when} = 1")

In [31]:
# Item restrictions in Super Metroid are monotone. This means that if item set A => item B reachable, then {C : C > A} => B reachable.
# This function computes a BDD which is equivalent to the input BDD, but it is monotonic over the prevs
def compute_non_monotonic(bdd):
    # Compute the set of non-monotonic edges
    # Use temp as the 'superset'
    bdd2 = context.let(prev_to_temp, bdd)
    # The set of all edges that are not usable with a larger set of items
    return context.let(temp_to_prev, context.exist(prevs, bdd & set_gte & ~bdd2))

def monotonize(bdd):
    non_mon = compute_non_monotonic(bdd)
    return bdd | non_mon

def count_items(bdd):
    counts = {}
    print(prevs)
    print(context.support(bdd))
    for i in context.support(bdd) & set(prevs):
        val = context.count(bdd & context.add_expr(f"{i} = 1"))
        if val != 0:
            counts[i] = val
    return counts

# Computes a CNF of a set of sets iff the set is monotone
# Update: Monotone Boolean Equivalence is co-NP Complete
# This function does not work as advertised.
def monotone_cnf(bdd):
    # List of lists
    # [[a,b],[c,d]] means [a or b] and [c or d]
    current_cnf = []
    no_nexts = context.exist(nexts, bdd)
    current_bdd = no_nexts
    current_cnf_bdd = context.true
    while current_cnf_bdd & current_bdd != current_cnf_bdd:
        current_count_outer = context.count(current_cnf_bdd & current_bdd)
        print(context.pick(current_cnf_bdd))
        print(current_count_outer)
        current_clause = []
        current_clause_bdd = context.false
        clause_cover = current_clause_bdd & current_bdd
        while clause_cover != current_bdd:
            current_count = context.count(clause_cover)
            print(current_count)
            print(context.support(current_bdd & ~current_clause_bdd) & set(prevs))
            item_counts = count_items(current_bdd & ~current_clause_bdd)
            print(item_counts)
            if len(item_counts) == 0:
                break
            i = max(item_counts, key = lambda x: item_counts[x])
            print(f"Choosing {i}")
            current_clause.append(i)
            current_clause_bdd |= context.add_expr(f"{i} = 1")
            clause_cover = current_clause_bdd & current_bdd
            next_count = context.count(clause_cover)
            print(next_count)
            print(f"Clause: {current_clause}")
        current_cnf.append(current_clause)
        current_cnf_bdd &= current_clause_bdd
        current_bdd = context.exist(current_clause, current_bdd)
        next_count_outer = context.count(current_cnf_bdd & current_bdd)
        print(context.support(current_bdd) & set(prevs))
        print("Outer:", next_count_outer)
        #assert(next_count_outer < current_count_outer)
        print(f"CNF: {current_cnf}")
    #assert current_cnf_bdd == no_nexts, "BDD was not monotone!"
    return current_cnf, current_cnf_bdd

In [32]:
prev_to_item = {f"{i}_prev": i for i in item_mapping}

In [33]:
def mk_set_eq(vars):
    eq = context.true
    for v in vars:
        eq &= context.add_expr(f"{v} = {prev_to_temp[v]}")
    return eq

def mk_set_lte(vars):
    lte = context.true
    for v in vars:
        lte &= context.add_expr(f"{v} <= {prev_to_temp[v]}")
    return lte

def mk_set_lt(vars):
    eq = mk_set_eq(vars)
    lte = mk_set_lte(vars)
    return lte & (~eq)

def minimize(bdd):
    bdd2 = context.let(prev_to_temp, bdd)
    lt = mk_set_lt(context.support(bdd))
    # All sets which are strict subsets of another set
    supers = context.let(temp_to_prev, context.exist(prevs, bdd & bdd2 & lt))
    return bdd & ~supers

# BDD should be prevs-only
# Requirements is a mapping from frozensets to frozensets
def mk_requirements(item, requirements):
    bdd = context.add_expr(f"{item}_prev = 0 & {item}_next = 1") & itemset_lattice
    bdd_p = context.exist(nexts, bdd)
    mins = minimize(monotonize(bdd_p))
    # Compute the conjunctive part - items required in every solution
    conj = set()
    for i in item_mapping:
        ibdd = context.add_expr(f"{i}_prev = 0")
        if (ibdd & mins).count() == 0:
            conj.add(i)
    #print(conj)
    # Compute the disjunctive part
    exists = [f"{c}_prev" for c in conj]
    mins2 = context.exist(exists, mins)
    n_disjs = context.count(mins2)
    #print(item, n_disjs)
    if n_disjs == 0:
        requirements[item] = frozenset()
        return
    # If there's more than one disj, then we have to make a disjunctive node
    if context.count(mins2) != 1:
        disj = set()
        # Make a node for each disjunctive solution
        for d in context.pick_iter(mins2):
            # We only care about required items
            dd = {k: v for k,v in d.items() if v == 1}
            # If the disj only has one node, that node can be a direct requirement
            if len(dd) == 1:
                [(k,v)] = list(dd.items())
                v == 1
                disj.add(prev_to_item[k])
            # Otherwise, we need to make a (potentially new) node for the combination
            #TODO: can potentially remove some things from the core if all disjunctive sets would require them
            # Conjunction over minimize(monotonize()) of the sets
            else:
                soln_i = set()
                for k,v in dd.items():
                    # If the item was needed
                    if v == 1:
                        soln_i.add(prev_to_item[k])
                fsoln_i = frozenset(soln_i)
                assert len(fsoln_i) > 1, f"{dd},  {len(dd)}"
                fname = frozenset(soln_i | set(["&"]))
                disj.add(fname)
                # Create a new node if needed
                if fname not in requirements:
                    requirements[fname] = fsoln_i
        # Make a disjunctive node for the disjunction
        fdisj = frozenset(disj)
        dname = frozenset(disj | set(["|"]))
        #print(item, dname)
        if dname not in requirements:
            requirements[dname] = fdisj
        requirements[item].add(dname)
    # Conj is the core; conj items are required by the current item
    requirements[item] = requirements[item] | conj

In [34]:
requirements = defaultdict(set)

In [35]:
mk_requirements("B", requirements)

In [36]:
requirements

defaultdict(set, {'B': {'M', 'MB'}})

In [37]:
def compute_requirements():
    # Compute the item requirements
    requirements = defaultdict(set)
    # For item_a to be required for item_b, we need two conditions:
        # - There is a reachable edge where the player has item_a and gets item_b
        # - There is no edge where the player does not have item_a and gets item_b
    for item_a, item_b in permutations(item_mapping.keys(), 2):
        get_b = helper.context.add_expr(f"{item_b}_prev = 0 & {item_b}_next = 1")
        get_with = helper.context.add_expr(f"{item_a}_prev = 1") & get_b
        get_without = helper.context.add_expr(f"{item_a}_prev = 0") & get_b
        if (trans_feasible & get_with).count() > 0 and (trans_feasible & get_without).count() == 0:
            # B requires A
            requirements[item_b].add(item_a)
    return requirements

In [38]:
def compute_requirements_dnf():
    requirements = defaultdict(set)
    for i in item_mapping:
        mk_requirements(i, requirements)
    return requirements

In [39]:
def simplify(requirements):
    requirements2 = defaultdict(set)
    for item_b, s in requirements.items():
        removes = set()
        for item_a in s:
            # item_a is not required if there exists another required item that also requires item_a
            for item_c in s:
                if item_c != item_a:
                    if item_c in requirements and item_a in requirements[item_c]:
                        removes.add(item_a)
        requirements2[item_b] = requirements[item_b] - removes
    return requirements2

# Prunes unnecessary "&" nodes (nodes with a single child after simplification)
def prune_ands(requirements):
    sfd = []
    for k,v in requirements.items():
        if "&" in k and len(v) == 1:
            print(f"Removing {k}")
            sfd.append(k)
            child = list(v)[0]
            # Find the parents and fix their requirements
            for k2, v2 in requirements.items():
                if k in v2:
                    v2 = v2 - frozenset({k})
                    v2 = v2 | frozenset({child})
                requirements[k2] = v2
    # Remove the nodes
    for k in sfd:
        del requirements[k]

#TODO: this is very inefficient since many sets will be compared repeatedly
def dnf_superset(dnf_a, dnf_b):
    id = "|"
    #print(f"Comparing {dnf_a} with {dnf_b}")
    assert id in dnf_a and id in dnf_b
    solns_a = dnf_a - set([id])
    solns_b = dnf_b - set([id])
    # A >= B if for each soln in A, there is a corresponding soln in B which is larger (requires at least as many items)
    for s_a in solns_a:
        if type(s_a) == str:
            s_a = set([s_a])
        found = False
        for s_b in solns_b:
            if type(s_b) == str:
                s_b = set([s_b])
            if s_a <= s_b:
                found = True
                continue
        if not found:
            return False
    return True

def in_reqs(item, requirements):
    id = "|"
    for k in requirements:
        if item == k:
            return True
        # If item >= req, then it is in the set (there is a more restrictive set)
        if type(item) == frozenset and id in item and type(k) == frozenset and id in k:
            return dnf_superset(item, k)

# Repeatedly remove unnecessary disjunctive nodes (dnodes without any forward edges) until a fixpoint
def prune_dnodes(requirements):
    removed = True
    while removed:
        removed = False
        sfd = []
        for k, v in requirements.items():
            # Only target disjunctive nodes
            if "|" in k or "&" in k:
                found = False
                # Find a node that requires k
                for k2, v2 in requirements.items():
                    if k in v2:
                        found = True
                if not found:
                    sfd.append(k)
        for k in sfd:
            del requirements[k]
        if len(sfd) > 0:
            removed = True

def simplify_dnf(requirements):
    requirements2 = defaultdict(set)
    for item_b, s in requirements.items():
        removes = set()
        for item_a in s:
            # item_a is not required if there exists another required item that also requires item_a
            for item_c in s:
                if item_c != item_a:
                    #if item_c in requirements and item_a in requirements[item_c]:
                    if in_reqs(item_c, requirements) and in_reqs(item_a, requirements[item_c]):
                        removes.add(item_a)
        requirements2[item_b] = requirements[item_b] - removes
    # Now remove unnecessary &s
    prune_ands(requirements2)
    prune_dnodes(requirements2)
    return requirements2

In [40]:
import graphviz

In [41]:
#requirements = compute_requirements()
#requirements = simplift(requirements)
#for i in item_mapping:
#    lattice.node(i) 
#for k,vs in requirements.items():
#    for v in vs:
#        lattice.edge(v, k)
#lattice.render("../../output/item_lattice")

In [47]:
requirements = compute_requirements_dnf()
requirements = simplify_dnf(requirements)

Removing frozenset({'&', 'SB', 'IB'})


In [48]:
# Clean up the requirements table
del requirements["Ceres_Ridley"]
del requirements["Bomb_Torizo"]
requirements["MB"] = frozenset(["START"])

In [50]:
from encoding import sm_global

In [52]:
itr = sm_global.item_translate

In [53]:
# Build the graph
lattice = graphviz.Digraph()
for k in requirements.keys():
    if type(k) == frozenset:
        assert not (set(["|", "&"]) < k), "Weird Node!"
        if "|" in k:
            lattice.node(str(k), label="|")
        elif "&" in k:
            lattice.node(str(k), label="&")
    elif k == "START":
        lattice.node("START", "START", color="blue", style="filled", shape="doubleoctagon")
    else:
        lattice.node(itr[k])
for k,vs in requirements.items():
    for v in vs:
        if k in itr:
            ks = sm_global.item_translate[str(k)]
        else:
            ks = str(k)
        if v in itr:
            vs = sm_global.item_translate[str(v)]
        else:
            vs = str(v)
        if "|" in k:
            lattice.edge(vs, ks, style="dashed")
        else:
            lattice.edge(vs, ks)
lattice.render("../../output/item_lattice")

'../../output/item_lattice.pdf'

In [45]:
timeb = time.time()

In [50]:
timeb - timea

8.51942777633667

In [46]:
#TODO compute requirements for every minor item using trans_feasible!
#TODO: dotted edges for | requirements
#TODO: what is going on with reserve tank requiring Varia?
#TODO: 

In [47]:
def convert(d):
    s = set()
    for k, v in d.items():
        if v == 1:
            s.add(k)
    return frozenset(s)

In [48]:
solns_s = set([convert(d) for d in solns])
solns2_s = set([convert(d) for d in solns2])
solns3_s = set([convert(d) for d in solns3])

NameError: name 'solns' is not defined

In [ ]:
extra = solns_s - solns2_s
extra

In [ ]:
for e in extra:
    for s in solns2_s:
        if e < s:
            print("LT")
            print(e)
            print(s)
            break

In [ ]:
from itertools import combinations

In [ ]:
def minimize_non_bdd(sets):
    sfd = set()
    sfdi = set()
    indices = set(range(len(sets)))
    for (i1, s1), (i2, s2) in combinations(enumerate(sets), 2):
        print(i1, i2)
        if s1 < s2:
            print(f"Removing set {i2}")
            print(s2 - s1)
            sfd.add(s2)
            sfdi.add(i2)
        if s2 < s1:
            print(f"Removing set {i1}")
            print(s1 - s2)
            sfd.add(s1)
            sfdi.add(i1)
    return sets - sfd, indices - sfdi

In [ ]:
#len(minimize_non_bdd(solns_s))

In [ ]:
minimize_non_bdd(solns_s)

In [ ]:
q = convert(context.pick(m_p & to_var("WB") & ~to_var("SB") & to_var("Crocomire")))
#q = convert(context.pick(m_p & ~to_var("IB") & to_var("SB") & ~to_var("WB") & to_var("Crocomire")))

In [ ]:
context.count(trans_feasible & ~to_var("IB") & to_var("SB") & ~to_var("WB") & to_var("Crocomire"))

In [ ]:
q

In [ ]:
for i, s in enumerate(solns_s):
    if s == q:
        print(i)